In [1]:
# | default_exp core.agent2tool

In [4]:
# | export
from typing import Dict, Any, List, Optional
from dataclasses import dataclass
from agentic.core.agent import Agent, AgentConfig
from agentic.tools.base import BaseTool 


In [6]:
# | export
@dataclass
class AgentToolConfig:
    """Configuration for agent-as-tool"""
    agent_name: str
    description: str
    parameters_schema: Dict[str, Any]
    timeout: int = 60
    max_iterations: int = 3


class AgentTool(BaseTool):
    """Wrapper to use an Agent as a Tool"""
    
    def __init__(self, agent: Agent, config: AgentToolConfig):
        self.agent = agent
        self.config = config
    
    @property
    def name(self) -> str:
        return self.config.agent_name
    
    @property
    def description(self) -> str:
        return self.config.description
    
    @property
    def parameters(self) -> Dict[str, Any]:
        return self.config.parameters_schema
    
    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute the agent with provided parameters"""
        try:
            # Prepare message for the agent
            message = self._prepare_agent_message(kwargs)
            
            # Run the agent
            result = self.agent.run(message)
            
            return {
                "success": True,
                "agent_name": self.config.agent_name,
                "result": result,
                "message": message
            }
            
        except Exception as e:
            return {
                "success": False,
                "agent_name": self.config.agent_name,
                "error": str(e)
            }
    
    def _prepare_agent_message(self, kwargs: Dict[str, Any]) -> str:
        """Prepare message for the agent based on parameters"""
        # Extract the main task/query
        task = kwargs.get("task") or kwargs.get("query") or kwargs.get("message")
        
        if not task:
            # Build task from all parameters
            param_strings = [f"{k}: {v}" for k, v in kwargs.items()]
            task = f"Process the following parameters: {', '.join(param_strings)}"
        
        return task


class AgentOrchestrator:
    """Orchestrates multiple agents working together"""
    
    def __init__(self):
        self.agents: Dict[str, Agent] = {}
        self.agent_tools: Dict[str, AgentTool] = {}
        self.workflows: Dict[str, List[str]] = {}
    
    def register_agent(self, name: str, agent: Agent, tool_config: Optional[AgentToolConfig] = None):
        """Register an agent and optionally create a tool wrapper"""
        self.agents[name] = agent
        
        if tool_config:
            agent_tool = AgentTool(agent, tool_config)
            self.agent_tools[name] = agent_tool
            
            # Add the agent tool to other agents
            for other_name, other_agent in self.agents.items():
                if other_name != name:
                    other_agent.add_tool(name, agent_tool.execute)
    
    def create_workflow(self, name: str, agent_sequence: List[str]):
        """Create a workflow of agents"""
        self.workflows[name] = agent_sequence
    
    def execute_workflow(self, workflow_name: str, initial_message: str) -> Dict[str, Any]:
        """Execute a predefined workflow"""
        if workflow_name not in self.workflows:
            return {"error": f"Workflow '{workflow_name}' not found"}
        
        sequence = self.workflows[workflow_name]
        results = []
        current_message = initial_message
        
        for agent_name in sequence:
            if agent_name not in self.agents:
                results.append({"error": f"Agent '{agent_name}' not found"})
                continue
            
            agent = self.agents[agent_name]
            result = agent.run(current_message)
            results.append({
                "agent": agent_name,
                "result": result
            })
            
            # Prepare message for next agent
            current_message = self._prepare_next_message(result, current_message)
        
        return {
            "workflow": workflow_name,
            "results": results,
            "final_result": results[-1] if results else None
        }
    
    def _prepare_next_message(self, previous_result: Dict[str, Any], original_message: str) -> str:
        """Prepare message for next agent in workflow"""
        content = previous_result.get("content", "")
        
        next_message = f"Previous agent result: {content}\n\n"
        next_message += f"Original task: {original_message}\n\n"
        next_message += "Continue processing based on the previous result."
        
        return next_message


# Predefined agent tool configurations
AGENT_TOOL_CONFIGS = {
    "code_analyzer": AgentToolConfig(
        agent_name="code_analyzer",
        description="Analyze code quality, structure, and suggest improvements",
        parameters_schema={
            "type": "object",
            "properties": {
                "code_path": {"type": "string", "description": "Path to code file or directory"},
                "analysis_type": {
                    "type": "string", 
                    "enum": ["quality", "security", "performance", "structure"],
                    "description": "Type of analysis to perform"
                }
            },
            "required": ["code_path"]
        }
    ),
    
    "data_processor": AgentToolConfig(
        agent_name="data_processor",
        description="Process and analyze data files",
        parameters_schema={
            "type": "object",
            "properties": {
                "data_path": {"type": "string", "description": "Path to data file"},
                "operation": {
                    "type": "string",
                    "enum": ["analyze", "clean", "transform", "visualize"],
                    "description": "Operation to perform on data"
                },
                "output_format": {"type": "string", "description": "Desired output format"}
            },
            "required": ["data_path", "operation"]
        }
    ),
    
    "documentation_generator": AgentToolConfig(
        agent_name="documentation_generator", 
        description="Generate documentation for code projects",
        parameters_schema={
            "type": "object",
            "properties": {
                "project_path": {"type": "string", "description": "Path to project directory"},
                "doc_type": {
                    "type": "string",
                    "enum": ["api", "user_guide", "readme", "technical"],
                    "description": "Type of documentation to generate"
                },
                "output_path": {"type": "string", "description": "Where to save documentation"}
            },
            "required": ["project_path", "doc_type"]
        }
    ),
    
    "test_generator": AgentToolConfig(
        agent_name="test_generator",
        description="Generate unit tests for code",
        parameters_schema={
            "type": "object", 
            "properties": {
                "code_path": {"type": "string", "description": "Path to code to test"},
                "test_framework": {
                    "type": "string",
                    "enum": ["pytest", "unittest", "jest", "mocha"],
                    "description": "Testing framework to use"
                },
                "coverage_target": {"type": "number", "description": "Target test coverage percentage"}
            },
            "required": ["code_path"]
        }
    )
}


def create_specialized_agents() -> Dict[str, Agent]:
    """Create a set of specialized agents"""
    agents = {}
    
    # Code Analyzer Agent
    code_analyzer_config = AgentConfig(
        name="code_analyzer",
        instructions="You are a code analysis expert. Analyze code for quality, security, and performance issues.",
        tools=["fs_read", "code_quality", "execute_bash"]
    )
    agents["code_analyzer"] = Agent(code_analyzer_config)
    
    # Data Processor Agent
    data_processor_config = AgentConfig(
        name="data_processor", 
        instructions="You are a data processing expert. Analyze, clean, and transform data efficiently.",
        tools=["fs_read", "fs_write", "code_interpreter"]
    )
    agents["data_processor"] = Agent(data_processor_config)
    
    # Documentation Generator Agent
    doc_generator_config = AgentConfig(
        name="documentation_generator",
        instructions="You are a technical writer. Create clear, comprehensive documentation.",
        tools=["fs_read", "fs_write", "doc_generator"]
    )
    agents["documentation_generator"] = Agent(doc_generator_config)
    
    # Test Generator Agent
    test_generator_config = AgentConfig(
        name="test_generator",
        instructions="You are a test automation expert. Generate comprehensive unit tests.",
        tools=["fs_read", "fs_write", "code_interpreter"]
    )
    agents["test_generator"] = Agent(test_generator_config)
    
    return agents
